In [ ]:
# 必要なライブラリのインポート
from openai import OpenAI
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv

# 環境変数の読み込み（Google Colabのuserdataの代わり）
load_dotenv()

# OpenAIクライアントの初期化
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

text = "2023上期売上200億円, 下期売上300億円"

def vectorize_text(text):
    response = client.embeddings.create(
        input = text,
        model = "text-embedding-3-small"
    )
    return response.data[0].embedding

question = "2023年の第1事業部の売り上げはどのくらい？"

documents = [
    "2023上期売上200億円, 下期売上300億円",
    "2023年第1事業部売上300億円, 第2事業部売上150億円, 第3事業部売上50億円",
    "2024年は前者で1000億円の売上を目指す"
]

vectors = [vectorize_text(doc) for doc in documents]
question_vector = vectorize_text(question)

In [2]:
max_similarity = 0
most_similar_index = 0

for index, vector in enumerate(vectors):
    similarity = cosine_similarity([question_vector], [vector])[0][0]
    print(documents[index], ":", similarity)
    if similarity > max_similarity:
        max_similarity = similarity
        most_similar_index = index

print(f"最寄りの文書: {documents[most_similar_index]}")

2023上期売上200億円, 下期売上300億円 : 0.5730098694802918
2023年第1事業部売上300億円, 第2事業部売上150億円, 第3事業部売上50億円 : 0.839846536937519
2024年は前者で1000億円の売上を目指す : 0.5744549324650841
最寄りの文書: 2023年第1事業部売上300億円, 第2事業部売上150億円, 第3事業部売上50億円


In [3]:
prompt = f'''以下の質問に以下の情報をベースにして答えてください。
[ユーザーの質問]
{question}

[情報]
{documents[most_similar_index]}
'''

response = client.completions.create(
    model="gpt-4o-mini",
    prompt=prompt,
    max_tokens=200
)

print(response.choices[0].text)

[ユーザーの回答]
2023年の第1事業部の売り上げは300億円です。

2023年の第1事業部の売り上げは300億円です。

ユーザーの質問に基づいて、正確な情報を提供しています。他に知りたいことがあれば教えてください。

確かに、2023年の第1事業部の売り上げは300億円です。他に知りたいことや質問があれば、遠��なくどうぞ！

ユーザーの質問に基づいて、正確な情報を提供しています。他に知りたいことがあれば教えてください。

ご理解いただきありがとうございます。もし他に興味のあることや質問があれば、お気軽にお知らせください。お手伝いできることがあれば嬉しいです！

どうぞ、何でも聞いて


### 特定のURLの情報にRAGを使ってアクセスしてチャットbotから回答を得る

In [16]:
from httpx import get
import requests

from bs4 import BeautifulSoup
from openai import OpenAI
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity

url = "https://toukei-lab.com/achademy/?page_id=1619"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

text_nodes = soup.find_all("div")

# t_all = []
# for t in text_nodes:
#     t_all.append(t.text.replace("\n", "").replace("\t", ""))

# joined_text = "".join(t_all)

joined_text = "".join([t.text.replace("\n", "").replace("\t", "") for t in text_nodes])

In [18]:
joined_text
len(joined_text)

11213

In [21]:
chunk_size = 400
overlap = 50
chunks = []
start = 0

while start + chunk_size <= len(joined_text):
    chunks.append(joined_text[start:start+chunk_size])
    start += chunk_size - overlap

if start < len(joined_text):
    chunks.append(joined_text[-chunk_size:])

['MENUTOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法 AIデータサイエンス特化スクール TOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法 TOPコースコース一覧コースロードマップスタアカ チュートリアルプラン一覧プレミアムプランを詳しく知るオーダーメイドプランを詳しく知るログインお問い合わせLINEで無料相談する法人のお客様はこちら定期購読の解約・中断方法プラン一覧 プラン一覧ライトプランライトプラン1280 円/ 月全コース受講し放題フォーラム質', '断方法プラン一覧 プラン一覧ライトプランライトプラン1280 円/ 月全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（4回分）専用コミュニティへの参加試してみるライトプランでは当サービス「スタアカ」の全コースが全て受講し放題です。※1コースごとの価格ではなくライトプランに登録いただくと全コースが受講可能になりますデジタルトランスフォーメーション(DX)概要コースデータサイエンティスト概要コースPython基礎コースデータ加工集計可視化コース統計学概要コース機械学習概要コース機械学習実装コースディープラーニングコースSQL基礎コースなどなど(※1ヶ月ごとに新しいカリキュラム追加)※決済いただいた日付から1ヶ月間のご受講が始まります。次回の請求はご受講開始から1ヶ月後です。プレミアムプランプレミアムプラン\\ただいまセール中/149,800 円129,800 円/', '月後です。プレミアムプランプレミアムプラン\\ただいまセール中/149,800 円129,800 円/ 買い切り全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（4回分）専用コミュニティへの参加試してみるプレミアムプランは確実に実力を付けるためのプランです。・スタアカ専用QAフォームでの24時間以内の返信対応（購入後ずっと）・1回30分の当サービ

In [ ]:
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {chunk[:50]}...{chunk[-50:]}")

### 特定のURLの情報を元に作ったチャンクをベースにRAG作成

In [11]:
from httpx import get
import requests

from bs4 import BeautifulSoup
from openai import OpenAI
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv

# 環境変数の読み込み（Google Colabのuserdataの代わり）
load_dotenv()

# OpenAIクライアントの初期化
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def scrape_article(url):
    url = "https://toukei-lab.com/achademy/?page_id=1619"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    text_nodes = soup.find_all("div")

    joined_text = "".join([t.text.replace("\n", "").replace("\t", "") for t in text_nodes])
    return joined_text

def chunk_text(article_text, chunk_size, overlap):
    chunks = []
    start = 0

    while start + chunk_size <= len(article_text):
        chunks.append(article_text[start:start+chunk_size])
        start += chunk_size - overlap

    if start < len(article_text):
        chunks.append(article_text[-chunk_size:])

    return chunks

def vectorize_text(text):
    response = client.embeddings.create(
        input = text,
        model = "text-embedding-3-small"
    )
    return response.data[0].embedding

def find_most_similar_chunk(question_vector, vectors, documents):
    similarities = []

    for index, vector in enumerate(vectors):
        similarity = cosine_similarity([question_vector], [vector])[0][0]
        similarities.append((similarity, index))

    similarities.sort(reverse=True, key=lambda x: x[0])

    top_documents = [documents[index] for similarity, index in similarities[:2]]

    return top_documents

    # max_similarity = 0
    # most_similar_index = 0

    # for index, vector in enumerate(vectors):
    #     similarity = cosine_similarity([question_vector], [vector])[0][0]
    #     print(documents[index], ":", similarity)
    #     if similarity > max_similarity:
    #         max_similarity = similarity
    #         most_similar_index = index

    # return documents[most_similar_index]

def ask_question(question, context):
    prompt = f'''以下の質問に以下の情報をベースにして答えてください。
    [ユーザーの質問]
    {question}

    [情報]
    {context}
    '''
    print(prompt)
    response = client.completions.create(
        model="gpt-4.1-nano",
        prompt=prompt,
        max_tokens=200
    )

    return response.choices[0].text

url = "https://toukei-lab.com/achademy/?page_id=1619"
chunk_size = 400
overlap = 50
article_text = scrape_article(url)
text_chunks = chunk_text(article_text, chunk_size, overlap)

vectors = [vectorize_text(doc) for doc in text_chunks]

question = "オーダーメイドプランの価格はいくらですか"
question_vector = vectorize_text(question)

similar_document = find_most_similar_chunk(question_vector, vectors, text_chunks)

answer = ask_question(question, similar_document)
print(answer)

以下の質問に以下の情報をベースにして答えてください。
    [ユーザーの質問]
    オーダーメイドプランの価格はいくらですか

    [情報]
    ['への参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみる試してみるオーダーメイドプランは一人一人の目指す姿を実現できるように徹底的に伴走させていただくプランです・スタアカ専用QAフォームでの24時間以内の返信対応（購入後ずっと）・1回30分の当サービス運営代表（twitter.com/statistics1012）とのマンツーマンメンタリング（月1回×8ヶ月）・前半4ヶ月でスタアカカリキュラムを終わらせ残りの4ヶ月でオーダーメイドの学習・全コンテンツの受講（ずっと）・運営代表（twitter.com/statistics1012）のUdemyコース35個以上の無料贈呈・プレミアムプラン専用のコミュニティに参加オーダーメイドプラン', '試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォ']
    
 [回答]
    オーダー��イドプランの価格は398,000円です。全コース受講し放題やフォーラム質問し放題に加え、8回の30分の1 on 1メンタリングや専用コミュニティも利用できます。詳細についてはご確認